# Convolutional Neural Network

In [7]:
# import related libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [8]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [9]:
# Hyper parameters
num_epochs = 5
num_classes = 100
batch_size = 100
learning_rate = 0.001

In [10]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../data',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)

test_dataset = torchvision.datasets.MNIST(root='../data',
                                         train=False,
                                         transform=transforms.ToTensor())

In [11]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

## torch.nn.Conv2d Learning

Pytorch official document API:

**class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)**

- Input(N, C_in, H, W)

- Output(N, C_out, H_out, W_out)

**Parameters:**

- in_channels(int) - 输入信号通道
- out_channels(int) - 卷积产生的通道
- kernel_size(int or tuple) - 卷积核的尺寸
- stride(int or tuple, optional) - 卷积步长
- padding(int or tuple, optional) - 输入的每条边补充0的层数
- dilation(int or tuple, optional) - 卷积核元素之间的间距
- groups(int, optional) - 从输入通道到输出通道的阻塞连接数
- bias(bool, optional) - 如果bias=True，添加偏置

**Example:**
```
# With square kernels and equal stride
m = nn.Conv2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# non-square kernels and unequal stride and with padding and dilation
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = (torch.randn(20, 16, 50, 100)
output = m(input)

"""
############# output ###########
tensor([[[[ 6.3524e-02, -1.2699e-01,  2.1988e-01,  ...,  2.8191e-01,
            2.3415e-01, -4.7665e-01],
          [-5.8807e-02,  1.1290e-01,  1.8089e-01,  ..., -5.2318e-01,
            1.4178e-01,  3.9162e-01],
          [ 1.9483e-01, -3.4395e-01,  5.6898e-01,  ..., -2.6373e-01,
           -1.3339e-01, -1.4554e-01],
"""

```

In [12]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        return out

model = ConvNet(num_classes).to(device)

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}"
                 .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 5.1718
Epoch [1/5], Step [200/600], Loss: 4.3976
Epoch [1/5], Step [300/600], Loss: 4.1541
Epoch [1/5], Step [400/600], Loss: 4.0046
Epoch [1/5], Step [500/600], Loss: 3.4848
Epoch [1/5], Step [600/600], Loss: 3.3392
Epoch [2/5], Step [100/600], Loss: 3.2931
Epoch [2/5], Step [200/600], Loss: 2.8741
Epoch [2/5], Step [300/600], Loss: 2.8902
Epoch [2/5], Step [400/600], Loss: 2.9294
Epoch [2/5], Step [500/600], Loss: 2.7296
Epoch [2/5], Step [600/600], Loss: 2.6284
Epoch [3/5], Step [100/600], Loss: 2.6850
Epoch [3/5], Step [200/600], Loss: 2.4596
Epoch [3/5], Step [300/600], Loss: 2.3567
Epoch [3/5], Step [400/600], Loss: 2.3069
Epoch [3/5], Step [500/600], Loss: 2.3588
Epoch [3/5], Step [600/600], Loss: 2.3820
Epoch [4/5], Step [100/600], Loss: 2.4221
Epoch [4/5], Step [200/600], Loss: 2.0558
Epoch [4/5], Step [300/600], Loss: 2.1766
Epoch [4/5], Step [400/600], Loss: 2.1796
Epoch [4/5], Step [500/600], Loss: 2.2883
Epoch [4/5], Step [600/600], Loss:

In [12]:
# Test the model
model.eval()     # # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print("Test accuracy of the model on the 10000 test images: {} %".format(100 * correct / total))


Test accuracy of the model on the 10000 test images: 30.54 %


The accuracy here is low, main reason is lack of training. The resolution to solve it, you could try to increase the epoch number.

In [13]:
# Save the model checkpoint
torch.save(model.state_dict(), "cnn.ckpt")